## Criando modelo MLP baseado no MNist  

https://docs.djl.ai/docs/demos/jupyter/tutorial/01_create_your_first_network.html  


### Preparation

In [1]:
%maven ai.djl:api:0.28.0
%maven ai.djl:basicdataset:0.28.0
%maven ai.djl:model-zoo:0.28.0
%maven ai.djl.mxnet:mxnet-engine:0.28.0
%maven org.slf4j:slf4j-simple:2.0.13

import java.nio.file.*;

import ai.djl.*;
import ai.djl.basicdataset.cv.classification.Mnist;
import ai.djl.ndarray.types.*;
import ai.djl.training.*;
import ai.djl.training.dataset.*;
import ai.djl.training.initializer.*;
import ai.djl.training.loss.*;
import ai.djl.training.listener.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.util.*;
import ai.djl.basicmodelzoo.cv.classification.*;
import ai.djl.basicmodelzoo.basic.*;

#### Ler dataset do Mnist


O Mnist já esta pré preparado no DJL. Os parametros de avaliação são construidos conforme o setSampling

In [2]:
int batchSize = 32;
Mnist mnist = Mnist.builder().setSampling(batchSize, true).build();
mnist.prepare(new ProgressBar());


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


### Criar Modelo   

In [3]:
Model model=Model.newInstance("mpl");
model.setBlock(new Mlp(28*28,10,new int[]{128,64}));

### Configurar treinamento

In [4]:
DefaultTrainingConfig config = new DefaultTrainingConfig(Loss.softmaxCrossEntropyLoss())
    //softmaxCrossEntropyLoss is a standard loss for classification problems
    .addEvaluator(new Accuracy()) // Use accuracy so we humans can understand how accurate the model is
    .addTrainingListeners(TrainingListener.Defaults.logging());

// Now that we have our training configuration, we should create a new trainer for our model
Trainer trainer = model.newTrainer(config);

### Inicializar treinamento

In [5]:
trainer.initialize(new Shape(1, 28 * 28));

### Treinar

In [6]:
// Deep learning is typically trained in epochs where each epoch trains the model on each item in the dataset once.
int epoch = 2;

EasyTrain.fit(trainer, epoch, mnist, null);

Training:    100% |████████████████████████████████████████| 
Training:    100% |████████████████████████████████████████| 


### Salvar modelo

In [7]:
Path modelDir = Paths.get("buildModel/mlpNotebook");
Files.createDirectories(modelDir);

model.setProperty("Epoch", String.valueOf(epoch));

model.save(modelDir, "mlp");

model

Model (
	Name: mpl
	Model location: /home/mertins/Desenvolvimento/Java/Clientes/Mertins/DLJ/Generic/src/main/jupyter/buildModel/mlpNotebook
	Data Type: float32
	Epoch: 2
)